# Class 2

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/kc_house_data.csv')
df.head()

In [ ]:
# displaying the first/last 11 columns
df.iloc[0:10,:11].head()
df.iloc[:,11:].head()

In [ ]:
# univariate analysis
df.describe()

# understand the output itself
sns.histplot(df['price'])
plt.hist(df['price'])

# cleaning a specific data point or value
df['bedrooms'] = df['bedrooms'].replace(33, 3) # replacing 33 with 3

In [ ]:
# finding the specific row
df.loc[15870]
# setting specific condition
sns.boxplot(df['bathrooms'])
df[df['bathrooms'] > 3.5].describe()
sns.boxplot(df[df['bathrooms'] > 3.5]['bathrooms'])
sns.boxplot(df[df['bathrooms'] > 3.5]['price'])
# winsorize outliers
values_to_replace_max = df[df['bathrooms'] > 3.5].index
df.loc[values_to_replace_max,'bathrooms'] = 3.5
df[df['bathrooms'] > 3.5]

In [ ]:
# boxplot with categories, x = category
sns.boxplot(x=df['view'], y=df['price'])

In [ ]:
pip install scipy
from scipy import stats
sns.boxplot(df['sqft_living'])
# another way to winsorize
df['sqft_living_winsorized'] = stats.mstats.winsorize(df['sqft_living'], limits=[0,0.01])
df[['sqft_living','sqft_living_winsorized']].sort_values('sqft_living', ascending=False)

In [ ]:
# heatmap
plt.figure(figsize = (20,20))
sns.heatmap(df.iloc[:,2:].corr(), annot=True)

# pairplot
sns.pairplot(df.iloc[:,2:])

# Class 3

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/kc_house_data.csv')
df.head()
# info() gives you structural information about the dataset
df.info()

In [ ]:
df['date'] = pd.to_datetime(df['date'])
# how volume changes over time (sort_index to sort with time)
df.groupby('date')['price'].count().sort_index().plot()
df.groupby('date')['price'].sum().sort_index().plot()
# looking at average overtime
df['price_per_sqft'] = df['price'] / df['sqft_living']
df.groupby('date')['price_per_sqft'].mean().sort_index().plot()

#identify the higher average price sold
df.groupby('date')['price_per_sqft'].mean().sort_values()

In [ ]:
#find the size of the particular high price house 
df.set_index('date').loc['2014-08-30']
# look at the usual price sold for the same size, identidy why the high price
df[(df['sqft_living'] >= 1000) & (df['sqft_living']<=1100)]['price'].mean()

In [ ]:
df[(df['sqft_living'] >= 1000) & (df['sqft_living']<=1100)]['price'].mean()
# look at the usual price sold for the same size, identidy why the high price
avg_price_zipcode = df.groupby(['zipcode'])['price_per_sqft'].mean().to_dict()
# average by zipcode
df['zip_price_per_sqft'] = df['zipcode'].map(avg_price_zipcode)
# creates a new column mapped with zipcode, showing the avergae price corresponded to the zipcode
df['compared_to_avg'] = df['price_per_sqft'] - df['zip_price_per_sqft']
sns.boxplot(df['compared_to_avg'])
#above/below the quantile are overpaid/underpaid 

In [ ]:
def value(x):
    if x > 0:
        return 'bad investment'
    elif x == 0:
        return 'fair investment'
    else:
        return 'good investment'
df['investment_type'] = df['compared_to_avg'].apply(value)
# apply single column tranformation using the function just created

In [ ]:
def value_v2(row):
    if row['compared_to_avg'] > 0 and row['grade'] > 10:
        return 'fair investment for a luxury property'
    elif row['compared_to_avg'] > 0 and row['grade'] < 10:
        return 'overpaid'
    elif row['compared_to_avg'] < 0 and row['grade'] > 10:
        return 'really good deal'
    else:
        return 'average deal'
df['deal_evaluation'] = df.apply(value_v2, axis = 1) 
# applied to the whole dataframe because using two or more columns in the function

In [ ]:
# Convert date column to datetime format
df['date'] = pd.to_datetime(df['date'])